# 팔머펭귄
> X를 주면 펭귄을 분류하는 문제

     Iter1.
- SVC같은 선형 분류 모델 하나 선정
- 그리드 서치 연습
- 파이프라인 구경

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("data_pipeline/penguins_size.csv")
df.sample(15)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
268,Gentoo,Biscoe,44.9,13.3,213.0,5100.0,FEMALE
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,FEMALE
41,Adelie,Dream,40.8,18.4,195.0,3900.0,MALE
267,Gentoo,Biscoe,50.5,15.9,225.0,5400.0,MALE
315,Gentoo,Biscoe,50.8,15.7,226.0,5200.0,MALE
279,Gentoo,Biscoe,50.4,15.3,224.0,5550.0,MALE
130,Adelie,Torgersen,38.5,17.9,190.0,3325.0,FEMALE
263,Gentoo,Biscoe,49.6,15.0,216.0,4750.0,MALE
148,Adelie,Dream,36.0,17.8,195.0,3450.0,FEMALE
151,Adelie,Dream,41.5,18.5,201.0,4000.0,MALE


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [4]:
df['species'].unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [5]:
df['island'].unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [6]:
df['sex'].unique()

array(['MALE', 'FEMALE', nan, '.'], dtype=object)

In [7]:
penguins=df.dropna()
penguins.reset_index(drop=True, inplace=True)

In [8]:
penguins[penguins['sex']=='.']

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
327,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,.


In [9]:
penguins.drop([327], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5692\1472391410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins.drop([327], inplace=True)


In [10]:
penguins['sex'].unique()

array(['MALE', 'FEMALE'], dtype=object)

## SVC로 Pipeline

In [11]:
penguins['species'] = penguins['species'].astype('category')
penguins['species_cod'] = penguins['species'].cat.codes

C:\Users\user\AppData\Local\Temp\ipykernel_5692\3177699378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species'] = penguins['species'].astype('category')
C:\Users\user\AppData\Local\Temp\ipykernel_5692\3177699378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species_cod'] = penguins['species'].cat.codes


In [12]:
penguins_feats=['island','culmen_length_mm','culmen_depth_mm',
                'flipper_length_mm','body_mass_g','sex']
penguins_labs=['species_cod']
X,y = penguins[penguins_feats].values, penguins[penguins_labs].values

## SVC를 사용한 학습 및 평가

In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix

In [14]:
n_feats = [1,2,3,4]
n_tras = Pipeline(steps=[('scaler', StandardScaler())]) #scaler는 StandardScaler()의 이름으로 선언
c_feats = [1,5]
c_tras = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',n_tras,n_feats),
        ('cat',c_tras,c_feats)
    ]
)

In [16]:
pipeline = Pipeline(steps=
                    [('preprocessor', preprocessor),('svc', SVC(probability=True))]
                    )

In [17]:
SVC_scores = []
SVC_fimp = []
SVC_predictions = 0
FOLDS = 5

In [18]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [19]:
for fold, (train_idx, valid_idx) in enumerate(skf.split(penguins[penguins_feats].values, 
                                                        penguins[penguins_labs].values)):
    print(f"Fold = {fold}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val_train, X_val, y_val_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
multi_model = pipeline.fit(X_val_train, y_val_train)
preds_valid = multi_model.predict(X_val)

acc = accuracy_score(y_val, preds_valid)
SVC_scores.append(acc)
test_preds = multi_model.predict(X_test)

Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [21]:
np.mean(SVC_scores)


0.9850746268656716

In [24]:
confusion_matrix(y_val, preds_valid)

array([[32,  0,  0],
       [ 1, 13,  0],
       [ 0,  0, 21]], dtype=int64)